In [ ]:
#this python notebook is for extracting data from labview measurement data

import matplotlib.pyplot as plt
import csv
import numpy as np
import sys
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
fs = 25600

file_path = filedialog.askopenfilename()

file = open(file_path)
t, s0, s1, s2, s3 = ([], [], [], [], [])

type(file)
for line in file:
    splitrow = line.rstrip().split("	")
    t.append(float(splitrow[0].replace(",", ".")))
    s0.append(float(splitrow[1].replace(",", ".")))
    s1.append(float(splitrow[2].replace(",", ".")))
    s2.append(float(splitrow[3].replace(",", ".")))
    s3.append(float(splitrow[4].replace(",", ".")))

In [ ]:


plt.plot(t,s0)